<a href="https://colab.research.google.com/github/jkrperson/volatility_prediction/blob/main/Volatility_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Download data (should be another python notebbook)

In [ ]:
!pip install wget

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9675 sha256=c04ee4dbc9bdc960c6d70f25cc39aaa82d498ef285a1ef5d2f9a14c60f499ece
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget


In [ ]:
import wget

In [ ]:
# begin = datetime.datetime(2021,3,1)
# end = datetime.datetime.now() - datetime.timedelta(days=2)
# ldf = []

# while begin <= end:
#   url = "https://data.binance.vision/data/spot/daily/klines/BTCBUSD/1m/BTCBUSD-1m-{date}.zip".format(date=begin.strftime("%Y-%m-%d"))
#   filename = wget.download(url, out=data_out_path)
#   # csv_file = shutil.unpack_archive(filename, csv_out_path)
#   # path = "/content/binance_data_csv/BTCBUSD-1m-{date}.csv".format(date=begin.strftime("%Y-%m-%d"))
#   # ldf.append(pd.read_csv(path, header=None))
#   begin = begin + datetime.timedelta(days=1)

# btcbusd_pd = pd.concat(ldf, axis=0, ignore_index=True)

## Extract data from CSV files

In [ ]:
!pip install arch

In [ ]:
import datetime
import shutil
import pandas as pd
import os

In [ ]:
# Create folders

data_out_path = "/content/binance_data_zip"
csv_out_path = "/content/binance_data_csv"
if not os.path.exists(data_out_path):
  os.makedirs(data_out_path)

if not os.path.exists(csv_out_path):
  os.makedirs(csv_out_path)

In [ ]:
import numpy as np
import pandas as pd
import os
import glob
import seaborn as sns

In [ ]:
# import necessary libraries


path = "/content/drive/MyDrive/Quant Dataset/binance_data_csv"
csv_files = glob.glob(os.path.join(path, "*.csv"))

ldf = []
for f in csv_files:
	
	# read the csv file
  ldf.append(pd.read_csv(f, header=None))
 
btcbusd_pd = pd.concat(ldf, axis=0, ignore_index=True)


## Arrange Data

In [ ]:
btcbusd_pd.columns = ["Open time", "Open", "High", "Low", "Close", "Volume", 
                      "Close Time", "Quote asset volume", "No. of Trades",
                      "Taker buy base asset volume", "Taker buy quote assete volume",
                      "Ignore"]

In [ ]:
btcbusd_pd["Timestamp"] = pd.to_datetime(btcbusd_pd["Open time"]*1e6)

In [ ]:
close_prices = btcbusd_pd["Close"]

In [ ]:
returns = close_prices.pct_change().dropna() * 100

## GARCH MODEL

In [ ]:
import arch

In [ ]:
train = returns[:60] * 10

In [ ]:
garch = arch.arch_model(train, vol='garch', p=1, o=0, q=1)
garch_fitted = garch.fit()

Iteration:      1,   Func. Count:      6,   Neg. LLF: 78766941.29094008
Iteration:      2,   Func. Count:     13,   Neg. LLF: 636.1436116825506
Iteration:      3,   Func. Count:     21,   Neg. LLF: 147.8982191301055
Iteration:      4,   Func. Count:     27,   Neg. LLF: 131.37574611112407
Iteration:      5,   Func. Count:     33,   Neg. LLF: 126.26650646583813
Iteration:      6,   Func. Count:     39,   Neg. LLF: 124.83828051995609
Iteration:      7,   Func. Count:     45,   Neg. LLF: 123.21543775961821
Iteration:      8,   Func. Count:     50,   Neg. LLF: 124.41163025011828
Iteration:      9,   Func. Count:     56,   Neg. LLF: 122.47148623535404
Iteration:     10,   Func. Count:     61,   Neg. LLF: 122.46556077310622
Iteration:     11,   Func. Count:     66,   Neg. LLF: 122.45960429220926
Iteration:     12,   Func. Count:     71,   Neg. LLF: 122.45960167849994
Iteration:     13,   Func. Count:     75,   Neg. LLF: 122.45960167862
Optimization terminated successfully    (Exit mode 0)
   

In [ ]:
forecast = garch_fitted.forecast(horizon=1, reindex=False)
variance_fore = forecast.variance.iloc[-1][0]

In [ ]:
forecast.variance

,h.01,h.02,h.03,h.04,h.05,h.06,h.07,h.08,h.09,h.10
60,1.297841,1.26397,1.230983,1.198856,1.167569,1.137097,1.107421,1.07852,1.050372,1.02296


In [ ]:
vol_fore = np.sqrt(variance_fore)
ann_vol = vol_fore * np.sqrt(252)/100
ann_vol

0.1208744469202321

## LSTM Model

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

### Prepare data

In [ ]:
from ipywidgets import IntProgress
from IPython.display import display

For this section, the target value should be changed to match the output of garch. 

In [ ]:
# ADD TRAIN TEST VAL

SEQ_LEN = 60

X = []
y = []

for x in range(len(returns) - SEQ_LEN):
  X.append(returns[x:x+SEQ_LEN]*100)
  y.append(returns[x:x+SEQ_LEN+1].var()*100) 
  if (x+1) % 10000 == 0:
    print("%d / %d"%(x+1, len(returns)-SEQ_LEN))

X = torch.from_numpy(np.array(X)).float()
y = torch.from_numpy(np.array(y)).float()

X = X.reshape((X.shape[0], X.shape[1], 1))
y = y.reshape((y.shape[0], 1))

10000 / 857265
20000 / 857265
30000 / 857265
40000 / 857265
50000 / 857265
60000 / 857265
70000 / 857265
80000 / 857265
90000 / 857265
100000 / 857265
110000 / 857265
120000 / 857265
130000 / 857265
140000 / 857265
150000 / 857265
160000 / 857265
170000 / 857265
180000 / 857265
190000 / 857265
200000 / 857265
210000 / 857265
220000 / 857265
230000 / 857265
240000 / 857265
250000 / 857265
260000 / 857265
270000 / 857265
280000 / 857265
290000 / 857265
300000 / 857265
310000 / 857265
320000 / 857265
330000 / 857265
340000 / 857265
350000 / 857265
360000 / 857265
370000 / 857265
380000 / 857265
390000 / 857265
400000 / 857265
410000 / 857265
420000 / 857265
430000 / 857265
440000 / 857265
450000 / 857265
460000 / 857265
470000 / 857265
480000 / 857265
490000 / 857265
500000 / 857265
510000 / 857265
520000 / 857265
530000 / 857265
540000 / 857265
550000 / 857265
560000 / 857265
570000 / 857265
580000 / 857265
590000 / 857265
600000 / 857265
610000 / 857265
620000 / 857265
630000 / 857265
6

In [ ]:
train_size = int(X.shape[0] * 0.50)
val_size = int(X.shape[0] * 0.25)
test_size = X.shape[0] - train_size - val_size

In [ ]:
train_X = X[:train_size]
train_y = y[:train_size]

val_X = X[train_size:train_size+val_size]
val_y = y[train_size:train_size+val_size]

test_X = X[train_size+val_size:]
test_y = y[train_size+val_size:]

### Define the LSTM Model

In [ ]:
class LSTMModel(nn.Module):
  def __init__(self, input_dim, hidden_dim, target_size):
    super(LSTMModel, self).__init__()
    self.hidden_dim = hidden_dim
    self.input_dim = input_dim

    self.lstm = nn.LSTM(input_dim, hidden_dim, batch_first=True)

    self.hidden2targ = nn.Linear(hidden_dim, target_size)

  def forward(self, sequence):
    lstm_out, _ = self.lstm(sequence)[-1]
    targ = self.hidden2targ(lstm_out)
    return targ

### Prepare for training

In [ ]:
import math

In [ ]:
if torch.cuda.is_available():
  dev = "cuda:0"
else:
  dev = "cpu"

device = torch.device(dev)

In [ ]:
model = LSTMModel(1, 100, 1)
loss_function = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.05)

In [ ]:
batch_size = 60
train_batches = math.ceil(train_X.shape[0] / batch_size)
val_batches = math.ceil(val_X.shape[0] / batch_size)
test_batches = math.ceil(test_X.shape[0] / batch_size)

In [ ]:
train_batches

7144

In [ ]:
f = IntProgress(min = 0, max=300)
display(f)

loss_track = []

model.to(device)

for epoch in range(300):
  train_epoch_loss = 0.0
  train_running_loss = 0.0

  for i in range(train_batches):
    model.zero_grad()

    x = train_X[i*batch_size:(i+1)*(batch_size)]
    label = train_y[i*batch_size:(i+1)*(batch_size)]
    label = label.reshape((1, label.shape[0], 1))

    x = x.to(device)
    label = label.to(device)

    targs = model(x)


    loss = loss_function(targs, label)
    loss.backward()
    optimizer.step()


    train_epoch_loss += targs.shape[0] * loss.sum().item()
    train_running_loss += loss.sum().item()

    # if i % 1000 == 999:
    #   print("\t[%d, %d] TRAIN RUN loss: %.6f" % (epoch+1, i+1, train_running_loss / 1000))
    #   train_running_loss = 0.0

  val_epoch_loss = 0.0
  val_running_loss = 0.0

  with torch.no_grad():
    for i in range(val_batches):
      model.zero_grad()

      x = val_X[i*batch_size:(i+1)*(batch_size)]
      label = val_y[i*batch_size:(i+1)*(batch_size)]
      label = label.reshape((1, label.shape[0], 1))

      x = x.to(device)
      label = label.to(device)

      targs = model(x)

      loss = loss_function(targs, label)
      val_epoch_loss += targs.shape[0] * loss.sum().item()

      val_running_loss += loss.sum().item()

      # if i % 1000 == 999:
      #   print("\t[%d, %d] VAL RUN loss: %.6f" % (epoch+1, i+1, val_running_loss / 1000))
      #   val_running_loss = 0.0


  print(epoch+1, "TRAIN: ", train_epoch_loss/train_batches, "\t VAL:", val_epoch_loss/val_batches)

  f.value += 1

IntProgress(value=0, max=300)

1 TRAIN:  107.13046801626018 	 VAL: 105.30029004291168
2 TRAIN:  108.05559339964164 	 VAL: 105.30029004291168
3 TRAIN:  108.05559339964164 	 VAL: 105.30029004291168
4 TRAIN:  108.05559339964164 	 VAL: 105.30029004291168
5 TRAIN:  108.05559339964164 	 VAL: 105.30029004291168


KeyboardInterrupt: ignored

In [ ]:
test_loss = 0.0

with torch.no_grad():
  for i in range(test_batches):
    model.zero_grad()
    
    x = test_X[i*batch_size:(i+1)*batch_size]
    label = test_y[i*batch_size:(i+1)*(batch_size)]
    label = label.reshape((1, label.shape[0], 1))

    x = x.to(device)
    label = label.to(device)

    targs = model(x)

    loss = loss_function(targs, label)

    test_loss += targs.shape[0] * loss.item()

print("TEST: ", test_loss/test_batches)

TEST:  45.134882556106625


In [ ]:
test = test_X[0:10]
test = test.to(device)
model(test)

tensor([[[ 0.4934],
         [-0.1559],
         [ 0.6292],
         [ 0.2146],
         [ 0.0599],
         [ 0.6040],
         [-0.0354],
         [-0.0979],
         [ 0.1685],
         [ 0.4877]]], device='cuda:0', grad_fn=<ViewBackward0>)

In [ ]:
test_y[0:10]

tensor([[0.1738],
        [0.1738],
        [0.1738],
        [0.1739],
        [0.1768],
        [0.1533],
        [0.1534],
        [0.1194],
        [0.1200],
        [0.1196]])

## DNN